In [1]:
#calculate distance 
import math
import random
import numpy as np

In [2]:
#useful functions from Day3
def calculate_total_energy(coordinates, box_length, cutoff):
    """
    Calculate the total energy of a set of particles using the Lennard Jones potential.
    
    Parameters
    ----------
    coordinates : list
        A nested list containing the x, y,z coordinate for each particle
    box_length : float
        The length of the box. Assumes cubic box.
    cutoff : float
        The cutoff length
    
    Returns
    -------
    total_energy : float
        The total energy of the set of coordinates.
    """
    
    total_energy = 0
    num_atoms = len(coordinates)

    for i in range(num_atoms):
        for j in range(i+1, num_atoms):
            # Calculate the distance between the particles - exercise.
            dist_ij = calculate_distance(coordinates[i], coordinates[j], box_length)

            if dist_ij < cutoff:
                # Calculate the pairwise LJ energy
                LJ_ij = calculate_LJ(dist_ij)

                # Add to total energy.
                total_energy += LJ_ij
    return total_energy

def read_xyz(filepath):
    """
    Reads coordinates from an xyz file.
    
    Parameters
    ----------
    filepath : str
       The path to the xyz file to be processed.
       
    Returns
    -------
    atomic_coordinates : np.array
        A two dimensional list containing atomic coordinates
    """
    
    with open(filepath) as f:
        box_length = float(f.readline().split()[0])
        num_atoms = float(f.readline())
        coordinates = f.readlines()
    
    atomic_coordinates = []
    
    for atom in coordinates:
        split_atoms = atom.split()
        
        float_coords = []
        
        # We split this way to get rid of the atom label.
        for coord in split_atoms[1:]:
            float_coords.append(float(coord))
        atomic_coordinates.append(float_coords)
    atomic_coordinates_np=np.array(atomic_coordinates)    
    return atomic_coordinates_np, box_length

def calculate_LJ(r_ij):
    """
    The LJ interaction energy between two particles.

    Computes the pairwise Lennard Jones interaction energy based on the separation distance in reduced units.

    Parameters
    ----------
    r_ij : float
        The distance between the particles in reduced units.
    
    Returns
    -------
    pairwise_energy : float
        The pairwise Lennard Jones interaction energy in reduced units.

    Examples
    --------
    >>> calculate_LJ(1)
    0

    """
    
    r6_term = math.pow(1/r_ij, 6)
    r12_term = math.pow(r6_term, 2)
    
    pairwise_energy = 4 * (r12_term - r6_term)
    
    return pairwise_energy

def calculate_distance(coord1, coord2, box_length=None):
    """
    Calculate the distance between two points. When box_length is set, the minimum image convention is used to calculate the distance between the points.

    Parameters
    ----------
    coord1, coord2 : list
        The coordinates of the points, [x, y, z]
    
    box_length : float, optional
        The box length

    Returns
    -------
    distance : float
        The distance between the two points accounting for periodic boundaries
    """
    distance = 0
        
    for i in range(3):
        hold_dist = abs(coord2[i] - coord1[i])
    
        if (box_length):    
            if hold_dist > box_length/2:
                hold_dist = hold_dist - (box_length * round(hold_dist/box_length))
        distance += math.pow(hold_dist, 2)

    return math.sqrt(distance)

## Add your group's tail correction function

def calculate_tail_correction(num_particles, box_length, cutoff):
    """
    The tail correction associated with using a cutoff radius.
    
    Computes the tail correction based on a cutoff radius used in the LJ energy calculation in reduced units.
    
    Parameters
    ----------
    num_particles : int
        The number of particles in the system.
    
    box_length : int
        Size of the box length of the system, used to calculate volume.
    
    cutoff : int
        Cutoff distance.
    
    Returns
    -------
    tail_correction : float
        The tail correction associated with using the cutoff.
    """
    
    brackets = (1/3*math.pow(1/cutoff,9)) - math.pow(1/cutoff,3)
    volume = box_length**3
    
    constant = ((8*math.pi*(num_particles**2))/(3*volume))
    
    tail_correction = constant * brackets
    
    return tail_correction

def calculate_pair_energy(coordinates, i_particle, box_length, cutoff):
    """
    Calculate the interaction energy of a particle with its environment (all other particles in the system)
    
    Parameters
    ----------
    coordinates : list
        The coordinates for all the particles in the system.
        
    i_particle : int
        The particle number for which to calculate the energy.
        
    cutoff : float
        The simulation cutoff. Beyond this distance, interactions are not calculated.
    
    box_length : float
        The length of the box for periodic bounds
        
    Returns
    -------
    e_total : float
        The pairwise interaction energy of the ith particles with all other particles in the system
    """
    
    e_total = 0.0
    #creates a list of the coordinates for the i_particle
    i_position = coordinates[i_particle]
    
    num_atoms = len(coordinates)
    
    for j_particle in range(num_atoms):
        
        if i_particle != j_particle:
            #creates a list of coordinates for the j_particle
            j_position = coordinates[j_particle]
            rij = calculate_distance(i_position, j_position, box_length)
            
            if rij < cutoff:
                e_pair = calculate_LJ(rij)
                e_total += e_pair
    
    return e_total

def accept_or_reject(delta_U, beta):
    """
    Accept or reject a move based on the Metropolis criterion.
    
    Parameters
    ----------
    detlta_U : float
        The change in energy for moving system from state m to n.
    beta : float
        1/temperature
    
    Returns
    -------
    boolean
        Whether the move is accepted.
    """
    if delta_U <= 0.0:
        accept = True
    else:
        #Generate a random number on (0,1)
        random_number = random.random()
        p_acc = math.exp(-beta*delta_U)
        
        if random_number < p_acc:
            accept = True
        else:
            accept = False
    return accept

In [3]:
# calculat distance for numpy array
def calculate_distance_np(coord1, coord2, box_length=None):
    """
    Calculate the distance between two points. When box_length is set, the minimum image convention is used to calculate the distance between the points.

    Parameters
    ----------
    coord1, coord2 : np.array
        The coordinates of the points, [x, y, z]
    
    box_length : float, optional
        The box length

    Returns
    -------
    distance : float
        The distance between the two points accounting for periodic boundaries
    """
    coord_dist=coord1-coord2
    if box_length:    
            coord_dist = coord_dist - (box_length * np.round(coord_dist/box_length))
    if coord_dist.ndim <2:
        #reshaping the array, make it have 2 dimensions
        coord_dist = coord_dist.reshape (1,-1)
    coord_dist=coord_dist**2
    dist_sum= coord_dist.sum(axis=1)
    distance = np.sqrt(dist_sum)
    return distance

In [4]:
#calculate LJ for num py
def calculate_LJ_np(r_ij):
    """
    The LJ interaction energies for an array of distances.

    Computes the pairwise Lennard Jones interaction energy based on the separation distance in reduced units. also considers cutoff

    Parameters
    ----------
    r_ij : np.arry
        The distance between the particles in reduced units.
    
    cutoff :
    
    Returns
    -------
    pairwise_energy : np.array
        The pairwise Lennard Jones interaction energy in reduced units.

    Examples
    --------
    >>> calculate_LJ(1)
    0

    """
    
    r6_term = (1./r_ij)** 6
    r12_term = (1./r_ij)** 12
    pairwise_energy = 4 * (r12_term - r6_term)

    return pairwise_energy

In [5]:
#calulate pair energy and total energy in numpy
def calculate_pair_energy_np(coordinates, i_particle, box_length, cutoff):
    """
    Calculate the interaction energy of a particle with its environment (all other particles in the system)
    
    Parameters
    ----------
    coordinates : np.array
        The coordinates for all the particles in the system.
        
    i_particle : int
        The particle number for which to calculate the energy.
        
    cutoff : float
        The simulation cutoff. Beyond this distance, interactions are not calculated.
    
    box_length : float
        The length of the box for periodic bounds
        
    Returns
    -------
    e_total : float
        The pairwise interaction energy of the ith particles with all other particles in the system
    """
    
    i_position_np =np.array(coordinates[i_particle])
    distances= calculate_distance_np(i_position_np,coordinates,box_length)
    #only account for distances less than cut off
    distances=distances[distances<cutoff]
    #eliminates point interaction with self
    new_distances=distances[distances != 0]
    e_pair = calculate_LJ_np(new_distances)
    e_total = e_pair.sum()
    
    return e_total

def calculate_total_energy_np(coordinates, box_length, cutoff):
    """
    Calculate the total energy of a set of particles using the Lennard Jones potential.
    
    Parameters
    ----------
    coordinates : np.array
        A nested list containing the x, y,z coordinate for each particle
    box_length : float
        The length of the box. Assumes cubic box.
    cutoff : float
        The cutoff length
    
    Returns
    -------
    total_energy : float
        The total energy of the set of coordinates.
    """
    pair_energies=[]
    for i in range(coordinates.shape[0]):
        #calculate_pair_energy_np will eliminate for cutoff and self interaction
        particle_pair_energy=calculate_pair_energy_np(coordinates, i, box_length, cutoff)
        pair_energies.append(particle_pair_energy)
    #turn list of pair energies into a numpy array
    pair_energies_np=np.array(pair_energies)
    #sum all pair energies to calculate total energy
    sum_energy=pair_energies_np.sum()
    # all pair-wise interactions are accounted for twice in the model
    total_energy=sum_energy/2
    return total_energy


In [6]:
coordinates, box_length = read_xyz('../../lj_sample_configurations/lj_sample_config_periodic1.txt')

In [15]:
assert calculate_distance(coordinates[0],coordinates[1],box_length)==calculate_distance(coordinates[0],coordinates[1],box_length)
assert calculate_LJ(2)==calculate_LJ_np(2)
assert math.isclose(calculate_pair_energy(coordinates,0,box_length,3),calculate_pair_energy_np(coordinates,0,box_length,3))
#assert calculate_total_energy(coordinates, box_length, cutoff)==calculate_total_energy_np(coordinates, box_length, cutoff)

In [13]:
print(calculate_pair_energy_np(coordinates,0,box_length,3))
print(calculate_pair_energy(coordinates,0,box_length,3))
calculate_pair_energy_np(coordinates,0,box_length,3)==calculate_pair_energy(coordinates,0,box_length,3)

-10.877945969430787
-10.877945969430792


False

In [9]:
coordinates.shape[0]

800

In [10]:
coordinates, box_length = read_xyz('../../lj_sample_configurations/lj_sample_config_periodic1.txt')

Monte Carlo Sim in NumPy

In [42]:

# Set simulation parameters
reduced_temperature = 0.9
num_steps = 50000
max_displacement = 0.1
cutoff = 3
    #how often to print an update
freq = 5000

# Calculated quantities
beta = 1 / reduced_temperature
num_particles = len(coordinates)

# Energy calculations
total_energy = calculate_total_energy_np(coordinates, box_length, cutoff)
print(total_energy)
total_correction = calculate_tail_correction(num_particles, box_length, cutoff)
print(total_correction)
total_energy += total_correction


for step in range(num_steps):
    # 1. Randomly pick one of the particles.
    random_particle = random.randrange(num_particles)
    
    # 2. Calculate the interaction energy of the selected particle with the system.
    current_energy = calculate_pair_energy_np(coordinates, random_particle, box_length, cutoff)
    
    # 3. Generate a random x, y, z displacement.
    rand_np=np.random.uniform(-max_displacement, max_displacement, 3)
    
    # 4. Modify the coordinate of Nth particle by generated displacements.
    coordinates[random_particle]= coordinates[random_particle] + rand_np
    
    # 5. Calculate the interaction energy of the moved particle with the system and store this value.
    proposed_energy = calculate_pair_energy_np(coordinates, random_particle, box_length, cutoff)
    delta_energy = proposed_energy - current_energy
    # 6. Calculate if we accept the move based on energy difference.
    accept = accept_or_reject(delta_energy, beta)
    # 7. If accepted, move the particle.
    if accept:
        total_energy += delta_energy
        #Move not accepted, roll back coordinates
    else:
        coordinates[random_particle]= coordinates[random_particle] - rand_np    
    
    # 8. Print the energy if step is a multiple of freq.
    if step % freq == 0:
        print(step, total_energy/num_particles)
    

-4311.590907210891
-198.4888837441566
0 -5.637599738693809
5000 -5.661055140694681
10000 -5.660293969044187
15000 -5.665666391479319
20000 -5.632746826988611
25000 -5.672777877507298
30000 -5.710481535239104
35000 -5.609199207416838
40000 -5.618781146094567
45000 -5.6749954544839225


In [30]:
current_energy = calculate_pair_energy_np(coordinates, 0, box_length, 3)
print(current_energy)

-10.877945969430787


In [31]:
rand_np=np.random.uniform(-max_displacement, max_displacement, 3)
print(rand_np)

[-0.0024092  -0.06449449 -0.03223072]


In [32]:
proposed_coord_np= coordinates[0] + rand_np
print(proposed_coord_np)

[-0.11504546  1.32059859 -0.91643423]


In [33]:
proposed_energy = calculate_pair_energy_np(proposed_coord_np, 0 , box_length, cutoff)
print(proposed_energy)

-0.1922288615156772
